Here there are a lot of choices for choosing the right model.
1. stsb-roberta-large: Ideal for semantic similarity
2. all-roberta-v1-large
3. all-mpnet-base-v2

In [59]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('all-roberta-large-v1', device = 'cpu')
model = SentenceTransformer('all-MiniLM-L12-v2', device = 'cpu')



### Random Testing

In [55]:

# Load both CSVs
csv1 = pd.read_csv("rupay_good_queries.csv", header=None, names=["text"])
csv2 = pd.read_csv("upi_good_queries.csv", header=None, names=["text"])

relevant_queries = pd.concat([csv1, csv2])['text'].tolist()


In [56]:

relevant_embeddings = model.encode(relevant_queries, convert_to_tensor=True)

def is_relevant(query, threshold=0.6):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, relevant_embeddings)
    max_sim = similarities.max().item()
    return True if max_sim >= threshold else False

test_queries = [
    "How do I register my startup?",
    "Tell me a joke.",
    "Can I apply for a business loan online?",
    "What's the weather today?",
    "How to find investors for my startup?",
    "get daily unique buyers this year"
]

for q in test_queries:
    print(f"Query: {q} → {is_relevant(q)}")


Query: How do I register my startup? → False
Query: Tell me a joke. → False
Query: Can I apply for a business loan online? → False
Query: What's the weather today? → False
Query: How to find investors for my startup? → False
Query: get daily unique buyers this year → True


In [57]:
def testing(filename):
	test_queries_df = pd.read_csv(filename, header=None, names=["text"])
	test_queries_df["Classification"] = test_queries_df["text"].apply(is_relevant)
	relevant_count = (test_queries_df["Classification"]).sum()
	not_relevant_count = len(test_queries_df) - relevant_count

	# Print results
	print(f"Relevant Queries: {relevant_count}")
	print(f"Not Relevant Queries: {not_relevant_count}")


In [58]:
testing("negative_queries.csv")
testing("simple.csv")

Relevant Queries: 0
Not Relevant Queries: 100
Relevant Queries: 197
Not Relevant Queries: 6
